In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
#import requests
import urllib
from datetime import datetime
from noticeEmail import noticeEMail
from selenium.common.exceptions import TimeoutException
import MDAnalysis as mda
import os
import errno
import numpy as np
starttime = datetime.now()
import numpy as np
import os.path
from pathlib2 import Path
from decimal import *
import mdtraj as md
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import rcParams
from scipy import optimize
import scipy.linalg
from mpl_toolkits.mplot3d import axes3d, Axes3D
import mpl_toolkits.mplot3d.art3d as art3d
import pytraj as pt
import MDAnalysis as mda
from pytraj import vector as va
import matplotlib.pylab as pylab
import re
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)
from scipy.interpolate import griddata
%matplotlib inline
import matplotlib.gridspec as gridspec
#filedir1 = '/gpfs/amarolab/abbyflabby/PAL/PALP_NaCl0.0/'
#filedir1 = '/gpfs/amarolab/abbyflabby/MIX/MIX234_NaCl0.4/'
filedir1 = '/scratch/abby/charmmgui/MIX/MIX1234_NaCl0.4/GROMACS/'
#surface_areas = ['19', '19.5', '20', '20.5', '21', '22', '24']

In [2]:
#variables to change
#['name', 'upper leaflet ratio', 'lower leaflet ratio', 'surface area']
waterHeight = "35"
xYLength = "60"
SA = '20.5'
i = SA
surface_areas=[SA]
fattyAcids = [
    #['pal', 4, 4, 19],
    ['palp', "4", "4", SA],
    ['laup', "1", "1", SA],
    #['lau', 1, 1, "19"]
    #['ste', 0, 0, 0],
    ['step', "3", "3", SA],
    #['myr', 0, 0, 0],
    ['myrp', "2", "2", SA]
]
saltConc = '0.4'
saltName = 'NaCl'
temperature = '298.15'
#these must be correct AND they must exist or script won't work!
fileDirectory = 'MIX/MIX1234_NaCl0.4/GROMACS/SA'+SA+'/'
saveFile = fileDirectory+'charmm-gui.tgz'
runFile = fileDirectory+'charmm-gui/gromacs/run.sh'
simulator = 'gmx_checked'
#SPECIFICALLY FOR MIX1 @ PH8.2:
#SIMULATOR OPTIONS
#amber = 'amb_checked'
#namd = 'namd_checked'
#gromacs = 'gmx_checked'
#must be gmail
email = 'acdommer@gmail.com'

In [3]:
def make_sure_path_exists(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise
make_sure_path_exists(fileDirectory)

In [4]:
try:    
    starttime = datetime.now()
    webpage = "http://charmm-gui.org/?doc=input/monolayer_only&step=1"
    driver = webdriver.PhantomJS('phantomjs-2.1.1-linux-x86_64/bin/phantomjs')
    driver.get(webpage)
    wait10 = WebDriverWait(driver, 10)
    wait600 = WebDriverWait(driver, 1000)
    water_thickness = driver.find_element_by_name("hetero_wdist")
    water_thickness.clear()
    water_thickness.send_keys(waterHeight)
    XY_length = driver.find_element_by_name("hetero_lx")
    XY_length.send_keys(xYLength)
    #click on triangles
    for i in range(1,14):
        driver.find_element(By.XPATH, '//img[@src="/images/basic/tri.png"]').click()
    #setting up an array of input Variables
    #[fatty acid name accepted by charmmgui, upper ratio, lower ratio, surface area per lipid]

    for i in fattyAcids:
        driver.find_element_by_name("lipid_ratio[upper]["+i[0]+"]").clear()
        driver.find_element_by_name("lipid_ratio[lower]["+i[0]+"]").clear()
        driver.find_element_by_name("lipid_ratio[area]["+i[0]+"]").clear()
        driver.find_element_by_name("lipid_ratio[upper]["+i[0]+"]").send_keys(i[1]) #upper
        driver.find_element_by_name("lipid_ratio[lower]["+i[0]+"]").send_keys(i[2]) #lower
        driver.find_element_by_name("lipid_ratio[area]["+i[0]+"]").send_keys(i[3]) #surface area
    #click show system info button
    driver.find_element_by_id('hetero_size_button').click()
    hetero_box = driver.find_element_by_id('hetero_size')
    wait10.until(EC.text_to_be_present_in_element((By.ID, 'hetero_size'), 'Calculated Number of Lipids'))
    driver.save_screenshot('screenie.png')
    #next page
    driver.find_element_by_id('nextBtn').click()
    #wait until page loads
    #after ten minutes, time out
    wait600.until(EC.url_contains('step=2'))
    #fill in salt concentration
    driver.find_element_by_id('ion_conc').clear()
    driver.find_element_by_id('ion_conc').send_keys(saltConc)
    salt_name = Select(driver.find_element_by_id('ion_type'))
    salt_name.select_by_visible_text(saltName)
    calculate_ions = driver.find_element_by_xpath('//input[@value="Calculate number of ions"]')
    calculate_ions.click()
    if saltConc == '0.0':
        driver.find_element_by_xpath('//input[@name="ion_checked"]').click()
    driver.find_element_by_id("nextBtn").click()
    #wait until page loads
    wait600.until(EC.url_contains('step=3'))
    driver.find_element_by_id("nextBtn").click()
    wait600.until(EC.text_to_be_present_in_element((By.ID, "fmonolayer_only"), "Assemble Generated Components"))
    driver.find_element_by_id("nextBtn").click()
    wait600.until(EC.url_contains('step=4'))
    driver.find_element_by_name(simulator).click()
    driver.find_element_by_name('temperature').clear()
    driver.find_element_by_name('temperature').send_keys(temperature)
    driver.find_element_by_id('nextBtn').click()
    wait600.until(EC.url_contains('step=5'))
    download_link_element = driver.find_element_by_css_selector('div#membrane_component a.download')
    #driver.save_screenshot('screenie.png')
    download_link = download_link_element.get_attribute('href')
    #print(download_link.get_attribute('href'))
    urllib.urlretrieve(download_link, saveFile)
except TimeoutException:
    runtime = datetime.now() - starttime
    subject = "CharmmGUI Error Notice (saddddd)"
    msg = "Mr. poopy butthole says your file "+fileDirectory+" has timed out in"+str(runtime)+"! You should probably go fix that"
    noticeEMail(starttime, email, msg, subject)

In [5]:
with open('untar.sh', 'a') as file:
    file.write('cd '+fileDirectory+'\n')
    file.write('tar zxvf charmm-gui.tgz\n')
! chmod 744 untar.sh
! ./untar.sh
! rm -rf untar.sh

charmm-gui/checkfft.py
charmm-gui/checkfft.str
charmm-gui/crystal_image.str
charmm-gui/lipid_lib.tar.gz
charmm-gui/membrane_lipid_restraint.gmx.str
charmm-gui/membrane_lipid_restraint.str
charmm-gui/membrane_lipid_restraint2.gmx.str
charmm-gui/membrane_lipid_restraint2.str
charmm-gui/step3_nlipids_lower.prm
charmm-gui/step3_nlipids_upper.prm
charmm-gui/step3_packing.inp
charmm-gui/step3_packing.out
charmm-gui/step3_packing.pdb
charmm-gui/step3_packing_head.crd
charmm-gui/step3_packing_head.pdb
charmm-gui/step3_packing_head.psf
charmm-gui/step3_packing_pol.str
charmm-gui/step3_size.inp
charmm-gui/step3_size.out
charmm-gui/step3_size.str
charmm-gui/step4.2_waterbox.crd
charmm-gui/step4.2_waterbox.inp
charmm-gui/step4.2_waterbox.out
charmm-gui/step4.3_ion.inp
charmm-gui/step4.3_ion.out
charmm-gui/step4.3_ion.pdb
charmm-gui/step4.3_neg.crd
charmm-gui/step4.3_pos.crd
charmm-gui/step4.3_pos.prm
charmm-gui/step4_components.str
charmm-gui/step4_lipid.crd
charmm-gui/step4_lipid.inp
charmm-gui/s

In [6]:
driver.close()
driver.quit()

In [7]:
pathToGMX = '/soft/gromacs/gromacs-5.0.4-DP/bin/gmx_d'

In [8]:
with open(runFile, 'w') as file:
    file.write('#Minimization with TIP3\n')
    file.write(pathToGMX+' grompp -f step6.0_minimization.mdp -o step6.0_minimization.tpr -c step5_charmm2gmx.pdb -p topol.top -maxwarn -1\n')
    file.write(pathToGMX+' mdrun -v -deffnm step6.0_minimization\n')
    file.write('#Equilibration with TIP3\n')
    file.write(pathToGMX+' grompp -f step6.1_equilibration.mdp -o step6.1_equilibration.tpr -c step6.0_minimization.gro -r step5_charmm2gmx.pdb -n index.ndx -p topol.top -maxwarn -1\n')
    file.write(pathToGMX+' mdrun -v -deffnm step6.1_equilibration\n')
    for f in range(2,7):
        file.write(pathToGMX+' grompp -f step6.'+str(f)+'_equilibration.mdp -o step6.'+str(f)+'_equilibration.tpr -c step6.'+str(f-1)+'_equilibration.gro -r step5_charmm2gmx.pdb -n index.ndx -p topol.top -maxwarn -1\n')
        file.write(pathToGMX+' mdrun -v -deffnm step6.'+str(f)+'_equilibration\n')

In [9]:
###rewrite call for TIP3 in topology to OPC4\
for i in surface_areas:
    with open(filedir1+'SA'+i+'/charmm-gui/gromacs/topol.top', 'r') as file:
        with open(filedir1+"SA"+i+'/charmm-gui/gromacs/topol_OPC4.top', 'w') as goose:
            for line in file:
                if line[10:25] == "toppar/TIP3.itp":
                    newline = line[:10]+"toppar/OPC4.itp"+line[25:]
                    goose.write(newline)
                elif line[10:25]=="toppar/charmm36":
                    newline = line[:10]+"toppar/charmm36_new.itp"+line[29:]
                    goose.write(newline)
                else:
                    goose.write(line.replace('TIP3', 'SOL '))
        file.close()
    file.close()
###Modify charmm36.itp to include the new water model info
for i in surface_areas:
    line1 = "[ atomtypes ]"+"\n"
    line2 = "      OW           0.0000      0.000     A    3.16655e-01          8.903586e-01"+"\n"
    line3 = "      HW           0.0000      0.000     A    0.00000e+00          0.00000e+00"+"\n"
    line4 = "      MW           0.0000      0.000     A    0.00000e+00          0.00000e+00"+"\n"
    line5 = "; name  at.num  mass    charge  ptype   sigma   epsilon ;       sigma_14        epsilon_14"

    with open(filedir1+'SA'+i+'/charmm-gui/gromacs/toppar/charmm36.itp', 'r') as file:
        with open(filedir1+'SA'+i+'/charmm-gui/gromacs/toppar/charmm36_new.itp', 'w') as goose:
            for line in file:
                if line[0:5] == "; nam":
                    goose.write(line)
                    goose.write(line2)
                    goose.write(line3)
                    goose.write(line4)
                else:
                    goose.write(line)
        goose.close()
    file.close()

    #with open('edit.sh', 'w') as file:
    #    file.write('cd '+filedir1+"SA"+i+"/charmm-gui/gromacs/toppar/\n")
    #    file.write('mv charmm36.itp charmm36_old.itp\n')
    #    file.write('mv charmm36_new.itp charmm36.itp\n')
    #    file.close()
    #! chmod 744 edit.sh
    #! ./edit.sh
    #! rm edit.sh
pdb = md.load(filedir1+"SA"+i+'/charmm-gui/step5_assembly.pdb')
water = pdb.top.select('water')
print(len(water))
add = len(water)/3
water_count = add+len(water)
print(water_count)
atom_count = len(pdb.top.select('all'))
print(atom_count)
total_indices = len(pdb.top.select('all'))+add
print(total_indices)
lipid_count = len(pdb.top.select('not water and not name SOD and not name CLA'))
print(lipid_count)

21489
28652
39375
46538
17784


## Start run file
<p>chmod 744 run.sh
<p>nohup ./run.sh &
<p>Can run the rest of the cells as soon as file finishes

## CANT DO THIS PART UNTIL TIP3P EQUILIBRATION IS FINISHED 

In [11]:
###this uses the convert script to 
for i in surface_areas:
    with open('3to4convert.sh','w') as file:
        file.write('gawk -f 3to4.awk '+filedir1+'SA'+i+'/charmm-gui/gromacs/step6.6_equilibration.gro > '+filedir1+'SA'+i+'/charmm-gui/gromacs/step6.7_equilibration.gro\n')
        file.write('cp OPC4.itp '+filedir1+'SA'+i+'/charmm-gui/gromacs/toppar')
    !chmod 744 3to4convert.sh
    !./3to4convert.sh

In [12]:
for i in surface_areas:
    #count how many atoms there are in newly edited equilibration.gro file then make new production file with new atom count
    #count = -3
    #with open(filedir1+"SA"+i+"/charmm-gui/gromacs/step6.6_equilibration_OPC4.gro", 'r') as file:
    #        for line in file:
    #            count += 1
    water = pdb.top.select('water')
    print(len(water))
    add = len(water)/3
    water_count = add+len(water)
    print(water_count)
    total_indices = len(pdb.top.select('all'))+add
    print(total_indices)
    ### create new gro file with appropriate number of atoms
    with open(filedir1+"SA"+i+"/charmm-gui/gromacs/step6.7_equilibration.gro", 'r') as file:
        with open(filedir1+"SA"+i+"/charmm-gui/gromacs/step6.7_production.gro", 'w') as goose:
            ct = 0
            for line in file:
                if ct != 1:
                    new_line = line.replace('TIP3', 'SOL ')
                    goose.write(new_line)
                    ct +=1 
                else: 
                    newline = str(total_indices)+'\n'
                    goose.write(newline)
                    ct +=1

21489
28652
46538


In [13]:
for i in surface_areas:
    with open(filedir1+'SA'+i+'/charmm-gui/gromacs/new.ndx', 'w') as goose:
        goose.write('[ MEMB ]\n')
        for k in range(1, total_indices+1):
            if k == lipid_count:
                goose.write('%5d' %k + '\n[SOL_ION]\n')
            elif k % 15 == 0:
                goose.write('%5d' %k + '\n')
            else: 
                goose.write('%5d' %k + ' ')
        goose.write('\n')
    #lalala

In [14]:
#i = '19-2'
### now we have to make the thing refer to new index rather than old index by changing the mdp.... 
### also change mdp for monolayer simulations
for i in surface_areas:
    with open('makeMinEq7.sh', 'w') as file:
        file.write('cp '+filedir1+'SA'+i+'/charmm-gui/gromacs/step6.0_minimization.mdp '+filedir1+'SA'+i+'/charmm-gui/gromacs/step7.0_minimization.mdp\n')
        for p in range(1, 7):
            file.write('cp '+filedir1+'SA'+i+'/charmm-gui/gromacs/step6.'+str(p)+'_equilibration.mdp '+filedir1+'SA'+i+'/charmm-gui/gromacs/step7.'+str(p)+'_equilibration.mdp\n')
    ! chmod 744 makeMinEq7.sh
    !./makeMinEq7.sh
    ! rm makeMinEq7.sh
    #with open(filedir1+'SA'+i+'/charmm-gui/gromacs/minimize.sh', 'w') as goose:
    #    goose.write('#Minimization\n')
    #    goose.write(pathToGMX+' grompp -f step7.0_minimization.mdp -o step7.0_minimization.tpr -c step6.7_production.gro -p topol.top -maxwarn -1\n')
    #    goose.write(pathToGMX+' mdrun -v -deffnm step7.0_minimization')
    with open(filedir1+'SA'+i+'/charmm-gui/gromacs/runOPC4.sh', 'w') as duck:
        duck.write('#!/bin/csh\n')
        duck.write('#Minimization OPC4\n')
        duck.write(pathToGMX+' grompp -f step7.0_minimization.mdp -o step7.0_minimization.tpr -c step6.7_production.gro -p topol_OPC4.top -maxwarn -1\n')
        duck.write(pathToGMX+' mdrun -v -deffnm step7.0_minimization\n')
        duck.write("#Equilibrate OPC4\n")
        duck.write(pathToGMX+' grompp -f step7.1_equilibration.mdp -o step7.1_equilibration.tpr -c step7.0_minimization.gro -r step7.0_minimization.gro -n new.ndx -p topol_OPC4.top -maxwarn -1\n')
        duck.write(pathToGMX+' mdrun -v -deffnm step7.1_equilibration\n')
        for f in range(2,7):
            duck.write(pathToGMX+' grompp -f step7.'+str(f)+'_equilibration.mdp -o step7.'+str(f)+'_equilibration.tpr -c step7.'+str(f-1)+'_equilibration.gro -r step7.0_minimization.gro -n new.ndx -p topol_OPC4.top -maxwarn -1\n')
            duck.write(pathToGMX+' mdrun -v -deffnm step7.'+str(f)+'_equilibration\n')
    #        for l, line in enumerate(file):
    ##            if l < 23:
    #                pass
    #            elif l < 38:
    #                new1 = line.replace('step6', 'step7')
    #                new2 = new1.replace('step5_charmm2gmx.pdb', 'step7.0_minimization.gro')
    ##                new3 = new2.replace('index.ndx', 'new.ndx')
    #                new4 = new3.replace('gmx', pathToGMX)
    #                new5 = new4.replace('topol', 'topol_OPC4')
    #                duck.write(new4)
    #            else:
    #                break

In [15]:
for i in surface_areas:
    with open(filedir1+'SA'+i+'/charmm-gui/gromacs/step7_production.mdp', 'r') as file:
        with open(filedir1+'SA'+i+'/charmm-gui/gromacs/step8.0_production.mdp', 'w') as goose:
            for line in file:
                if line[0:7] == "pcoupl ":
                    new_line = "pcoupl                  = no"
                    goose.write(new_line+'\n')
                    goose.write("DispCorr                = EnerPres\n")
                elif line[0:6] == "nsteps":
                    new_line = "nsteps                  = 5000000"
                    goose.write(new_line+"\n")
                else:
                    goose.write(line)

In [16]:
for i in surface_areas:
    with open(filedir1+'SA'+i+'/charmm-gui/gromacs/README', 'r') as file:
        with open(filedir1+'SA'+i+'/charmm-gui/gromacs/production.sh', 'w') as goose:
            goose.write('#!/bin/csh\n')
            for q, line in enumerate(file):
                if q < 39:
                    pass
                elif q < 54:
                    new1 = line.replace('gmx', pathToGMX)
                    new2 = new1.replace('step7_production.mdp', 'step8.0_production.mdp')
                    new3 = new2.replace('step7', 'step8')
                    new4 = new3.replace('step6', 'step7')
                    new5 = new4.replace('index', 'new')
                    new6 = new5.replace('1000', '5000000')
                    new7 = new6.replace('topol','topol_OPC4')
                    goose.write(new7)
                else:
                    break